In [6]:
# Bring in all dependencies
import pandas as pd
import requests
import numpy as np
import scipy as sp
from statistics import mode
from babel.numbers import format_number, format_decimal, format_percent, format_currency, decimal
from decimal import localcontext, Context, ROUND_UP

In [4]:
########## Read in data file ##########
path = 'static/resources/clean_data.csv'
clean_data_df = pd.read_csv(path,index_col = False)

In [7]:
# Get the last (greatest) timepoint for each stock
# Get the most common overall volitility score for each stock
max_date_df = clean_data_df.groupby(['Symbol']).agg({'Date':'max', 'Stock Volitility Score': mode})

# Merge max date df with the original dataframe to get the stock info at the last timepoint
merge_df = pd.merge(clean_data_df, max_date_df, on='Symbol')

#pull only the max time points from the df for most recent stock data
user_table = merge_df.loc[merge_df['Date_x']==merge_df['Date_y']]

# Keep only columns to display to user in app
user_table = user_table[['Symbol', 'Name', 'Sector', 'Close', 'Dividends', 'Stock Volitility Score_y']]

# Rename columns to match user experience
user_table = user_table.rename(columns={'Symbol': 'Ticker', 'Name': 'Company', 'Sector': 'Industry', 'Close': 'Price', 'Dividends': 'Dividends', 'Stock Volitility Score_y': 'Risk'})

# Add ROI column
user_table['ROI'] = user_table['Dividends'] / user_table['Price']

# relocate ROI column
col_to_move = user_table.pop('ROI')
user_table.insert(5, 'ROI', col_to_move)

# translate Risk score to high med low text
for score in user_table['Risk']:
    if score <= 6:
        user_table['Risk'] = 'Low Risk'
    elif score <= 9:
        user_table['Risk'] = 'Moderate Risk'
    else:
        user_table['Risk'] = 'High Risk'

# format price, dividends, ROI
for i,row in user_table.iterrows():
    user_table['Price'] = format_currency(round(row['Price'],3), 'USD', '¤ #,##0.000', locale='en_US', currency_digits=False, decimal_quantization=False)
    user_table['Dividends'] = format_currency(round(row['Dividends'], 4), 'USD', '¤ #,##0.0000', locale='en_US', currency_digits=False, decimal_quantization=False)
    user_table['ROI'] = format_percent(round(row['ROI'], 4), locale='en_US', decimal_quantization=False)

In [8]:
######### Save user table as csv for flask matching #########
user_table.to_csv('static/resources/user_table.csv',index=False)